In [2]:
from google.colab import drive
drive.mount('/content/gdrive') # 點擊網址，選擇 Google 帳號登入，然後將授權碼貼回輸入框中

Mounted at /content/gdrive


In [3]:
!ln -fs /content/gdrive/My\ Drive/Colab\ Notebooks/ /app  #執行一次就可，否則會有錯誤訊息
%cd /app

/content/gdrive/My Drive/Colab Notebooks


In [8]:
# BCH 63,45,18
# BCH_encode(Message)  產生BCH編碼 
# BCH_noise(Message)  對 Message 加入高斯雜訊
# BCH_abs(Message)   對 Message 取絕對值
# BCH_rand(Message,N)  亂數產生N個錯誤

import random
import math
import numpy as np
import random
from scipy import special

C=63
M=51
P=C-M
GX=5433

class BCH:
  def toArr(_message):
    # 陣列轉換
    BIN=[]
    for i in range(0,C):
      BIN.append((_message>>i)&1)
    BIN.reverse()
    return BIN

  def encode(_message): # BCH編碼， _message 訊息
    # 初始化
    _message = _message << P
    ans = 0
    MG = 1
    # 編碼
    for i in range(C):
      if(_message & (1<<i)):
        ans = ans ^ MG
      MG = MG << 1
      if(MG >= 2**P):
        MG = MG ^ GX 
    _message = _message|ans
    return BCH.toArr(_message)

  def sym(_message): # 計算症狀
    ans = _message & ((1 << P) - 1)
    _GX = GX & ((1 << P) - 1)
    for i in range(P,M+P):
      if(_message & (1<<i)):
        ans = ans ^ _GX
      _GX = _GX << 1
      if(_GX & 1<<P):
        _GX = _GX ^ GX 
    return BCH.toArr(ans)

  def noise(_message, _r): # 加入高斯雜訊， _message 訊息 、 _r DB值
    BIN=_message
    sigma = math.sqrt((10**-(_r/10))/2)
    for i in range(len(_message)):
      if(BIN[i]==0):
        BIN[i]=1.0+float(np.random.normal(0, sigma, 1))
      else:
        BIN[i]=-1.0+float(np.random.normal(0, sigma, 1))
    return BIN

  def rand(_message, _value): # 加入雜訊， _message 訊息 、 _value 雜訊數量
    data = _message
    status = 0;
    ans=[]
    # print(bin(_message))
    for i in range(_value):
      bit = int(random.uniform(0,63))
      ans.append(bit)
      if ((status & (1<<bit)) == 0):
        data = data ^ (1<<(bit))
        status = status ^ (1<<(bit));
    out_data = data
    ans.append(out_data)
    return ans

  def abs(_message): # 取絕對值
    BIN=_message
    for i in range(len(_message)):
      if(BIN[i]<=0):
        BIN[i]=1
      else:
        BIN[i]=0
    return BIN

  def arr_to_bin(_arr): # 陣列轉數值 
    ans=0
    for i in range(len(_arr)):
      ans = ans | (b[i] << i)
    return ans

  def Channel(_message,_w): #通道雜訊
    Eb = 4
    No = 1
    a = [0 for _ in range(0,C)]
    p = 0.5 - special.erf(math.sqrt(2.0 * Eb / No) / math.sqrt(2.0)) / 2
    BIN = []
    Y = []
    for i in range(0,C):
      BIN.append((_message >> i) & 1)
    BIN.reverse()
    while (_w > 0):
      n = int(random.uniform(0,C))
      if (a[n] == 0):
        a[n] = 1
        _w = _w - 1
    for i in range(0,C):
      U = random.random()
      F = p + (1 - p) * U
      if (a[i] == 1):
        F = p * U
      y = math.sqrt(Eb) + math.sqrt(No) * special.erfinv(2 * F - 1)
      if (BIN[i] == 1):
        Y.append(-y)
      else:
        Y.append(y)
    return Y,a


In [ ]:
# f = open('Training_one_1_to_50000.txt','w')#35184372088832
# for i in range(1,50000):
#   f.write(str(BCH.rand(BCH.encode(i),2))+'\n')
# f.close()

In [9]:
y=[]
t=[]
for i in range(1,1000):
  a,b=BCH.Channel(0,2)
  a.append(BCH.sym(BCH.arr_to_bin(a)))
  y.append(a)
  t.append(b)

In [10]:
print(y[1])
print(t[1])

[1.7215716251880235, 1.6866451920265104, -0.06702030917260693, 0.9249664938024809, 1.8985150268768505, 1.2827464629167635, 2.1153405143578627, 1.570056772602358, 2.6359583294689, 1.60633072509941, 2.4714648566232285, 0.8744503159583046, 2.984082048463712, 2.2342152396420265, -0.3157812174265353, 0.3043173575937024, 2.0901478881952937, 2.5253460178622347, 1.6445857981321308, 2.7308940125827315, 2.6714104296905905, 1.7775233150932244, 2.1924329336823325, 1.557188237863269, 2.2763424618911157, 2.3007396487328586, 2.4398540814209215, 1.7624129805655995, 2.6720399305242037, 2.257859899854881, 1.4433490489960106, 3.1038146624403624, 1.8007486663886079, 0.03654575074172328, 1.5872813582964584, 1.6863912704571749, 2.2374112860431543, 2.4788845762497664, 1.0571396996746811, 2.0281477419804275, 1.1348364825990909, 2.336440820067085, 1.8483759117678131, 1.8902400525260257, 2.582708719049659, 2.6737397706654646, 2.331410277549009, 1.2088819612647674, 1.7432429744312679, 1.3100805476463386, 0.30407